# Algorithm 23: Backbone Update

The Backbone Update converts the single representation to updates for the backbone frames (rigid body transformations).

## Algorithm Pseudocode

![BackboneUpdate](../imgs/algorithms/BackboneUpdate.png)

## Source Code Location
- **File**: `AF2-source-code/model/folding.py`
- **Function**: `FoldIteration` (within Structure Module)
- **Lines**: 263-280

## Purpose

The Backbone Update:
1. Takes the single representation as input
2. Predicts quaternion rotation and translation updates
3. Applies these updates to the current backbone frames

In [ ]:
import numpy as np

np.random.seed(42)

In [ ]:
def quat_to_rotation_matrix(q):
    """Convert quaternion to rotation matrix."""
    w, x, y, z = q[..., 0], q[..., 1], q[..., 2], q[..., 3]
    
    # Normalize quaternion
    norm = np.sqrt(w**2 + x**2 + y**2 + z**2 + 1e-8)
    w, x, y, z = w/norm, x/norm, y/norm, z/norm
    
    R = np.zeros(q.shape[:-1] + (3, 3))
    R[..., 0, 0] = 1 - 2*(y**2 + z**2)
    R[..., 0, 1] = 2*(x*y - w*z)
    R[..., 0, 2] = 2*(x*z + w*y)
    R[..., 1, 0] = 2*(x*y + w*z)
    R[..., 1, 1] = 1 - 2*(x**2 + z**2)
    R[..., 1, 2] = 2*(y*z - w*x)
    R[..., 2, 0] = 2*(x*z - w*y)
    R[..., 2, 1] = 2*(y*z + w*x)
    R[..., 2, 2] = 1 - 2*(x**2 + y**2)
    
    return R


def backbone_update(s, T_current):
    """
    Backbone Update - Algorithm 23.
    
    Predicts and applies frame updates from single representation.
    
    Args:
        s: Single representation [N_res, c_s]
        T_current: Current backbone frames (R, t)
            R: [N_res, 3, 3] rotation matrices
            t: [N_res, 3] translations
    
    Returns:
        Updated frames (R_new, t_new)
    """
    N_res, c_s = s.shape
    R_curr, t_curr = T_current
    
    print(f"Backbone Update")
    print(f"  Single repr: [{N_res}, {c_s}]")
    print(f"  Current frames: R {R_curr.shape}, t {t_curr.shape}")
    
    # Step 1: Predict quaternion and translation updates
    # Linear: c_s -> 6 (quaternion: 4, translation: 3, but simplified)
    W = np.random.randn(c_s, 6) * 0.01
    b = np.zeros(6)
    
    updates = s @ W + b  # [N_res, 6]
    
    # Split into quaternion update (4 values) and translation update (3 values)
    # For simplicity, we use 3+3 split
    delta_rot = updates[:, :3]  # Simplified rotation update
    delta_t = updates[:, 3:]    # Translation update
    
    # Step 2: Convert rotation update to matrix
    # Using quaternion: (1, delta_rot) normalized
    quat = np.zeros((N_res, 4))
    quat[:, 0] = 1.0  # w = 1
    quat[:, 1:] = delta_rot * 0.1  # Small rotation updates
    
    delta_R = quat_to_rotation_matrix(quat)  # [N_res, 3, 3]
    
    # Step 3: Compose with current frame
    # R_new = R_curr @ delta_R
    R_new = np.einsum('nij,njk->nik', R_curr, delta_R)
    
    # t_new = t_curr + R_curr @ delta_t
    t_new = t_curr + np.einsum('nij,nj->ni', R_curr, delta_t)
    
    print(f"  Updated frames: R {R_new.shape}, t {t_new.shape}")
    
    return (R_new, t_new)

In [ ]:
# Test
N_res, c_s = 32, 384

s = np.random.randn(N_res, c_s)

# Initialize frames as identity
R_init = np.tile(np.eye(3), (N_res, 1, 1))  # [N_res, 3, 3]
t_init = np.zeros((N_res, 3))  # [N_res, 3]

print("Test Backbone Update")
print("="*50)

R_new, t_new = backbone_update(s, (R_init, t_init))

# Verify rotation matrices are valid
print(f"\nVerification:")
print(f"  R orthogonal (R @ R.T ≈ I): {np.allclose(R_new[0] @ R_new[0].T, np.eye(3), atol=0.1)}")
print(f"  det(R) ≈ 1: {np.allclose(np.linalg.det(R_new[0]), 1.0, atol=0.1)}")

## Source Code Reference

```python
# From AF2-source-code/model/folding.py

# In FoldIteration:
# Jumper et al. (2021) Suppl. Alg. 23 "BackboneUpdate"
quaternion_update = common_modules.Linear(
    6, initializer='zeros', name='quaternion')(
        act)
quaternion_update = quaternion_update * self.config.position_scale
affine = quat_affine.QuatAffine.from_tensor(backb_to_global.to_tensor())
affine = affine.pre_compose(quaternion_update)
backb_to_global = r3.rigids_from_quataffine(affine)
```